## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config_gmap import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rio Gallegos,AR,2021-06-06 06:21:27,-51.6226,-69.2181,35.65,69,0,11.50,clear sky
1,1,Rikitea,PF,2021-06-06 06:21:49,-23.1203,-134.9692,76.05,83,72,15.90,light rain
2,2,Suntar,RU,2021-06-06 06:21:49,62.1444,117.6319,52.77,73,100,2.91,overcast clouds
3,3,Bethel,US,2021-06-06 06:21:49,41.3712,-73.4140,76.19,73,1,0.00,clear sky
4,4,Ancud,CL,2021-06-06 06:21:50,-41.8697,-73.8203,46.22,83,96,4.76,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df = preferred_cities_df.dropna()

preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,2021-06-06 06:21:49,-23.1203,-134.9692,76.05,83,72,15.90,light rain
3,3,Bethel,US,2021-06-06 06:21:49,41.3712,-73.4140,76.19,73,1,0.00,clear sky
5,5,Samarai,PG,2021-06-06 06:21:50,-10.6167,150.6667,78.33,87,94,16.28,light rain
17,17,Touros,BR,2021-06-06 06:21:56,-5.1989,-35.4608,74.30,91,52,11.23,broken clouds
19,19,Kapaa,US,2021-06-06 06:20:35,22.0752,-159.3190,77.32,76,40,1.99,scattered clouds
20,20,Saint-Philippe,RE,2021-06-06 06:21:57,-21.3585,55.7679,79.21,80,60,2.53,light rain
22,22,Ayapel,CO,2021-06-06 06:21:58,8.3137,-75.1398,75.07,96,98,2.37,overcast clouds
31,31,Grand Gaube,MU,2021-06-06 06:22:01,-20.0064,57.6608,79.38,73,70,1.99,broken clouds
57,57,Isangel,VU,2021-06-06 06:22:13,-19.5500,169.2667,77.11,65,75,15.93,broken clouds
59,59,Carnarvon,AU,2021-06-06 06:22:13,-24.8667,113.6333,73.47,60,90,12.66,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                158
City                   158
Country                158
Date                   158
Lat                    158
Lng                    158
Max Temp               158
Humidity               158
Cloudiness             158
Wind Speed             158
Current Description    158
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# No empty rows

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,76.05,light rain,-23.1203,-134.9692,
3,Bethel,US,76.19,clear sky,41.3712,-73.4140,
5,Samarai,PG,78.33,light rain,-10.6167,150.6667,
17,Touros,BR,74.30,broken clouds,-5.1989,-35.4608,
19,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,
20,Saint-Philippe,RE,79.21,light rain,-21.3585,55.7679,
22,Ayapel,CO,75.07,overcast clouds,8.3137,-75.1398,
31,Grand Gaube,MU,79.38,broken clouds,-20.0064,57.6608,
57,Isangel,VU,77.11,broken clouds,-19.5500,169.2667,
59,Carnarvon,AU,73.47,overcast clouds,-24.8667,113.6333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.count()

City                   158
Country                158
Max Temp               158
Current Description    158
Lat                    158
Lng                    158
Hotel Name             158
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,76.05,light rain,-23.1203,-134.9692,Pension Maro'i
3,Bethel,US,76.19,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
5,Samarai,PG,78.33,light rain,-10.6167,150.6667,Nuli Sapi
17,Touros,BR,74.30,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
19,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
20,Saint-Philippe,RE,79.21,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
22,Ayapel,CO,75.07,overcast clouds,8.3137,-75.1398,Hotel Colsama
31,Grand Gaube,MU,79.38,broken clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
57,Isangel,VU,77.11,broken clouds,-19.5500,169.2667,Tanna Lodge
59,Carnarvon,AU,73.47,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))